<a href="https://colab.research.google.com/github/ankit-rathi/Tradevesting_v2/blob/main/myStocks_Portfolio_Analysis_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr

In [2]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
import os
project_path = '/content/drive/My Drive/tradevesting'
os.chdir(project_path)

import warnings
warnings.filterwarnings('ignore')

from google.colab import data_table

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Import the function from the script
import myStocks_Portfolio_Analysis_v2

# Call the function to get the DataFrame
df_portfolio_features = myStocks_Portfolio_Analysis_v2.get_portfolio_features_v2()
cols = ['Symbol', 'AvgCost', 'Close', 'FTT', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'XIRR%', 'RRR Ind', 'Criteria', 'Strategy', 'Category']
df_portfolio_features = df_portfolio_features[cols]

Run date time (IST): 2024-07-29 14:53:46
-------------------
qualified stocks: 80
with latest results: 78
still star stocks: 46
-------------------
Initial Investment:  88.92 L
CY Investment:  1.02 C
Reserve:  1.56 L
Current:  1.15 C
-------------------
Today PnL: 96.11 K (0.85%)
Current PnL: 8.19 L (8.03%)
CY Booked + Current PnL: 12.58 L (12.33%)
-------------------
Total profit:  11.49 L
Total loss:  -3.30 L
-------------------
Total Booked + Current PnL: 25.69 L (28.89%)
Total Booked PnL: 17.50 L (19.68%)
Curr Year Booked PnL: 4.39 L (3.83%)
Prev Year Booked PnL: 13.11 L (14.74%)
Est FTT:  1.62 C
Est FTT PnL: 47.33 L (41.29%)
-------------------
Est LTT:  2.07 C
Est LTT PnL: 92.13 L (80.38%)
Deployed:  89.02 L
Current:  1.15 C
CAGR/XIRR %: 29.55%


In [5]:
# all portfolio
df_portfolio_features = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
data_table.DataTable(df_portfolio_features, include_index=False)

,Symbol,AvgCost,Close,FTT,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,XIRR%,RRR Ind,Criteria,Strategy,Category
19,CAMS,2424.02,4257.95,4450.00,106449.0,45848.0,4801.0,5.61,75.66,4.51,83.58,83.0,9.55,AVR,BTT,MISC
49,MARICO,530.08,684.10,725.00,136820.0,30804.0,8182.0,0.64,29.06,5.98,36.77,57.0,3.76,AV40,BTT,FMCG
22,DABUR,523.35,641.00,710.00,112175.0,20589.0,12070.0,1.35,22.48,10.76,35.66,28.0,1.71,AV40,BTT,FMCG
39,INFY,1436.24,1868.40,1997.00,194314.0,44945.0,13369.0,-0.56,30.09,6.88,39.04,50.0,3.36,AV40,BTT,IT
30,HCLTECH,1349.00,1633.15,1821.00,122486.0,21311.0,14086.0,-0.13,21.06,11.50,34.99,169.0,1.51,AV40,BTT,IT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18,CAMPUS,297.39,310.10,631.85,143576.0,5884.0,148974.0,-0.27,4.27,103.76,112.47,NaN,0.04,AVSP,ATH,FOOTWEAR
10,BANDHANBNK,227.40,217.15,400.00,179583.0,-8477.0,151209.0,12.81,-4.51,84.20,75.90,-21.0,-0.06,AVSP,NTT,BANKS
31,HDFCBANK,1569.25,1609.60,2170.00,437811.0,10975.0,152446.0,-0.53,2.57,34.82,38.28,4.0,0.07,AV40,BTT,BANKS
69,TANLA,969.42,962.00,2007.31,159692.0,-1232.0,173521.0,0.11,-0.77,108.66,107.06,-8.0,-0.01,AAR,ATH,IT


In [ ]:
# AAR analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AAR') ]
data_table.DataTable(tmp_df, include_index=False)

In [ ]:
# AVSP analysis
tmp_df = df_portfolio_features[(df_portfolio_features['Criteria'] == 'AVSP') ]
data_table.DataTable(tmp_df, include_index=False)